In [1]:
import pandas as pd
import multiprocessing

import re
import os
os.chdir(r'D:\Хакатон')
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, precision_recall_curve
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LinearRegression

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sunny\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_excel("ИКРБС_2020-2024_гг..xlsx")

In [3]:
df.fillna('Отсутствует', inplace=True)

In [4]:
df.drop(["№ п/п", "Регистрационный номер"],axis=1,inplace=True)

In [5]:
df['text_data'] = df['Наименование отчёта'] + ' ' + df['Реферативное описание отчета'] + ' ' + df['Ключевые слова']



In [6]:
df = df.drop_duplicates()

In [7]:
df = df.sample(frac=0.1)


In [8]:
df

,Наименование отчёта,Реферативное описание отчета,Полученные (ожидаемые) результаты и их возможная практическая значимость (применимость),Ключевые слова,ГРНТИ,Первая рубрика ГРНТИ,text_data
80027,Разработка прототипа системы автоматизированно...,Задачи первого этапа НИОКР:1.Разработать модул...,Отсутствует,"система рекомендаций, сканирование стоп, автом...",20.15.13: Информационные службы на предприятия...,27.21.21: Геометрическое исследование объектов...,Разработка прототипа системы автоматизированно...
29543,Разработка регламентов лабораторного культивир...,Объектом настоящего исследования являются разл...,Проведено изучение влияния различных компонент...,Steinernema feltiae; симбиотические бактерии; ...,68.37.13: Методы защиты растений от вредителей...,68.37.29: Вредители растений и борьба с ними,Разработка регламентов лабораторного культивир...
6975,ТЕХНОЛОГИЯ ДИАГНОСТИКИ ИЗМЕНЕНИЯ МОРФОФУНКЦИОН...,Данный этап исследование проведен на лаборатор...,В ходе данного исследования получены следующие...,КРОВЕНОСНЫЕ СОСУДЫ; СЕРДЦЕ; СТАРЕНИЕ; ЭНДОТЕЛИ...,76.29.30: Кардиология и ангиология; 34.41.02: ...,76.29.30: Кардиология и ангиология,ТЕХНОЛОГИЯ ДИАГНОСТИКИ ИЗМЕНЕНИЯ МОРФОФУНКЦИОН...
28295,Молекулярные основы редких неврологических заб...,В ходе выполнения первого этапа проекта нами б...,В ходе выполнения первого этапа проекта нами б...,мутации витамин-связывающих белков; болезнь ша...,34.15.43: Молекулярная нейробиология; 34.15.63...,34.15.63: Молекулярная фармакология и токсикол...,Молекулярные основы редких неврологических заб...
70825,Новые гадолиний-содержащие наночастицы в качес...,Выявленные закономерности свидетельствуют о це...,Отсутствует,"КОНТРАСТНЫЕ АГЕНТЫ, ЯМР ТОМОГРАФИЯ, КОМПЛЕКСЫ ...",31.15.37: Химия коллоидов. Дисперсные системы;...,31.15.37: Химия коллоидов. Дисперсные системы,Новые гадолиний-содержащие наночастицы в качес...
...,...,...,...,...,...,...,...
39695,НАУЧНАЯ БИБЛИОТЕКА РЕГИОНА В УСЛОВИЯХ МЕНЯЮЩЕЙ...,"Научная библиотека, являющаяся объектом исслед...",В 2022 г. на основе обследования факторов соци...,научная библиотека; регион; научная школа; нау...,13.31.15: Организация и управление в области б...,13.31.07: Теория и методология библиотечного дела,НАУЧНАЯ БИБЛИОТЕКА РЕГИОНА В УСЛОВИЯХ МЕНЯЮЩЕЙ...
37896,ОТЧЕТ О НАУЧНО-ИССЛЕДОВАТЕЛЬСКОЙ РАБОТЕ по тем...,В 2022 году были проведены исследования по алг...,Получена явная формула для группы компонент св...,"теория инвариантов, теория кодирования, универ...",27.17.19: Кольца и модули; 27.17.21: Структуры...,27.17.17: Группы,ОТЧЕТ О НАУЧНО-ИССЛЕДОВАТЕЛЬСКОЙ РАБОТЕ по тем...
19769,"Современные методы теории динамических систем,...",Цель работы: развитие современной теории динам...,"Исследована динамика диска конечной толщины, к...",ХАОС; ТОПОЛОГИЯ; ИСКРИВЛЕННЫЕ ПРОСТРАНСТВА; НЕ...,30.15.19: Устойчивость и стабилизация движения...,"30.15.15: Механика точки, системы и твердого тела","Современные методы теории динамических систем,..."
57822,"Изучение причин, клинических особенностей и пу...",Более 90% пациентов изучаемой выборки регистра...,Изучив структуру и распространенность факторов...,РЕГИСТР ПО АРТЕРИАЛЬНОЙ ГИПЕРТЕНЗИИ; РЕЗИСТЕНТ...,76.29.30: Кардиология и ангиология,76.29.30: Кардиология и ангиология,"Изучение причин, клинических особенностей и пу..."


In [9]:
sentence_example = df.iloc[1]["text_data"]
tokens = word_tokenize(sentence_example, language="russian")
tokens_without_punctuation = [i for i in tokens if i not in string.punctuation]
russian_stop_words = stopwords.words("russian")
tokens_without_stop_words_and_punctuation = [i for i in tokens_without_punctuation if i not in russian_stop_words]
snowball = SnowballStemmer(language="russian")
stemmed_tokens = [snowball.stem(i) for i in tokens_without_stop_words_and_punctuation]


In [10]:
print(f"Исходный текст: {sentence_example}")
print("-----------------")
print(f"Токены: {tokens}")
print("-----------------")
print(f"Токены без пунктуации: {tokens_without_punctuation}")
print("-----------------")
print(f"Токены без пунктуации и стоп слов: {tokens_without_stop_words_and_punctuation}")
print("-----------------")
print(f"Токены после стемминга: {stemmed_tokens}")
print("-----------------")

Исходный текст: Разработка регламентов лабораторного культивирования нематод, обеспечивающих выход суспензии жизнеспособных инвазионных личинок с заданными биологическими свойствами (агрессивность по отношению к целевым объектам. Биотестирование культур нематод на целевых объектах в лабораторных, вегетационных и полевых экспериментах. Оценка биологической эффективности опытных образцов биоинсектицида на различных сельскохозяйственных культурах. Объектом настоящего исследования являются различные виды и линии энтомопаразитических нематод, как потенциальные агенты для контроля численности и вредоносности различных насекомых-фитофагов, важных вредителей сельскохозяйственных культур. Цель второго этапа: Разработка регламентов лабораторного культивирования нематод, обеспечивающих выход суспензии жизнеспособных инвазионных личинок с заданными биологическими свойствами (агрессивность по отношению к целевым объектам. Биотестирование культур нематод на целевых объектах в лабораторных, вегетацио

In [11]:
snowball = SnowballStemmer(language="russian")
russian_stop_words = stopwords.words("russian")

def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language="russian")
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

In [12]:
tokenize_sentence(sentence_example)

['разработк',
 'регламент',
 'лабораторн',
 'культивирован',
 'нематод',
 'обеспечива',
 'выход',
 'суспенз',
 'жизнеспособн',
 'инвазион',
 'личинок',
 'зада',
 'биологическ',
 'свойств',
 'агрессивн',
 'отношен',
 'целев',
 'объект',
 'биотестирован',
 'культур',
 'нематод',
 'целев',
 'объект',
 'лабораторн',
 'вегетацион',
 'полев',
 'эксперимент',
 'оценк',
 'биологическ',
 'эффективн',
 'опытн',
 'образц',
 'биоинсектицид',
 'различн',
 'сельскохозяйствен',
 'культур',
 'объект',
 'настоя',
 'исследован',
 'явля',
 'различн',
 'вид',
 'лин',
 'энтомопаразитическ',
 'нематод',
 'потенциальн',
 'агент',
 'контрол',
 'числен',
 'вредоносн',
 'различн',
 'насекомых-фитофаг',
 'важн',
 'вредител',
 'сельскохозяйствен',
 'культур',
 'цел',
 'втор',
 'этап',
 'разработк',
 'регламент',
 'лабораторн',
 'культивирован',
 'нематод',
 'обеспечива',
 'выход',
 'суспенз',
 'жизнеспособн',
 'инвазион',
 'личинок',
 'зада',
 'биологическ',
 'свойств',
 'агрессивн',
 'отношен',
 'целев',
 'объек

In [13]:
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['text_data'].values.astype('U'))


In [20]:
X

<8171x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 913022 stored elements in Compressed Sparse Row format>

In [14]:
y = df['ГРНТИ']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [16]:
model = SGDClassifier(max_iter=10000) 
model.fit(X_train, y_train)

SGDClassifier(max_iter=10000)

In [17]:
y_pred = model.predict(X_test)

In [18]:

accuracy = accuracy_score(y_test, y_pred)
print('Точность:', accuracy)

Точность: 0.11376146788990826


In [19]:
# Новый текст
new_text = "электроника"

# Преобразование нового текста в вектор признаков
new_vector = vectorizer.transform([new_text])

# Получение предсказания рубрики ГРНТИ
predicted_grnti = model.predict(new_vector)

print("Предсказанная рубрика ГРНТИ:", predicted_grnti)

Предсказанная рубрика ГРНТИ: ['47.35.31: Лазеры']
